# Inspecting and cleaning the MAC data

In [1]:
# get macc data and imports
from wrangle import get_mac_data
import pandas as pd
from prepare_module import summarize
import geopandas

In [2]:
# get mac data
df = get_mac_data()

## initial examination

In [ ]:
summarize(df)

#### Takeaways:
* nulls:
    * **CLOSEDDATE**: expected, implies that they ar not closed, will invesitgate to ensure there aren't any errors
    * **TOWNSHIP__C**: investigate why, may need some domain knowledge about indianapolis metro area townships, where they lie 
    * [indy gis township information](https://data.indy.gov/datasets/75f6558b7a074edb8fa5fa9b0d1fc867/explore)
    * [townships throughout the state](https://www.stats.indiana.edu/maptools/maps/boundary/townships_2000/townships_statewide.pdf)
    * **COUNCIL_DISTRICT__C**: investigate why, I think there may be a connection with the council districts nulls and the township nulls 
        * Note: there may be council redistricting and township redistrictinhg that may be impacting this 
    * **SOURCE_ID**: what is the source id in the first place? id it necessary for the analysis? What the the source Id correspond with? is each source id unique or are they connected to something meaningful?
    * **CITY__C**: The city can be found by plotting the address on a map.
        1. Find out if python has something for addresses in the united states
        1. If not, get the geometries for eact objectid
        1. Get the geometries for cities in indiana
        1. Find out where each objectid geometry falls and fill in the CITY__C nulls with the new data
* Features/Variables:
    * ORIGIN: see if there is a relationship between origin and the time a service case is handled
    * STATUS:
        * Question: What proportion of each *category* is open/closed?
        * Question: What proportion of each *city* is open/closed?
        * Question: What proportion of each *township* is open/closed?
        * Question: What proportion of each *council district* is open/closed?
        * Question: What proportion of each *zip code* is open/closed?
    * Dates:
        * CLOSEDDATE: 
            * What are some reasons for the closed dates having (over 100) on one second? Is there a system wide update that occurs?
            * Is there a connection between the createddate and the closeddate?
        * CREATEDDATE: 
            * Why are there mulitple CREATEDDATE entries with the same second? Are there multiple categories of service frequests oin one call that are all inputed? Are these duplicates?
    * COUNCIL_DISTRICT__C
        * See how many districts there are and which districts have the best response times and worst response times
    * ZIP__C
        1. Make sure this is the zip code
        1. Get a csv file with the zipcode geometries in the states of indiana
        1. Merge the csv file with this data using the zipcode as the point of merging
        1. Use this to create a map of who has good time, who has bad response time, and whjo has great response times
     * CITY__C : 
        1. Change indianapolisq mispelling
        1. Investigate the 'NONE'
        1. Determine which has the fastest turnaround for non-emergency calls
        1. Does the Mayor Action thing apply to the other cities??
    * INCIDENT_ADDRESS__C
        * See if we can plot the incident addresses and identify the city for each 
        * Check to see whay some incidetn addressess have so many counts int he data
    * SUBCATEGORY__C:
        * Consolidate subcategories
        * If after consolidation , place all low couont subcategories into a new subcaegory called 'low_count_subcategories'
    * KEYWORD__C:
        1. Order all the key words alphabetically to check for duplicateswith similar spelling, see if it is intentional
        1. Within duplicates, ensure the subcategories are similary to the duplucatedones
    * SOURCE_ID__C
        * not very clear how this works. is there a list of source_id signifieers or a key that shows the meaning and significance of this information?
    * CASENUMBER:
        * Whya re there duplicate case numbers?


#### Actions:
* nulls:
    * CLOSEDDATE: Initially, I will fill all the closeddates with a number far out into the future that is not possible to represent unclosed
    *  I will drop all nulls for now and later go and fill in the missing information with other data sources


## Addressing nulls and changing column names

In [9]:


# change columsn names to lower, remove the __c and that's it
col_rename = {}

for col in df.columns:
    col_rename[col] = col.lower().replace('__c', '')

df.rename(col_rename, axis=1, inplace = True)

In [70]:
# changing multiple columns to date time dtype
print(df.columns)

df[['createddate','lastmodifieddate', 'closeddate']] = df[['createddate','lastmodifieddate', 'closeddate']].apply(pd.to_datetime)

df.dtypes

Index(['source_id', 'keyword', 'subcategory', 'incident_address', 'township',
       'city', 'zip', 'council_district', 'createddate', 'lastmodifieddate',
       'closeddate', 'status', 'origin'],
      dtype='object')


ValueError: Cannot mix tz-aware with tz-naive values, at position 15

In [36]:
# changing the closeddate nulls to a date that hasnt come
df[df.closeddate.isna() == True]

df.closeddate.fillna('9999',  inplace=True)

In [41]:
df.closeddate.value_counts()

pd.to_datetime(df.closeddate, errors='coerce').value_counts()

closeddate
2018-05-01 16:32:02+00:00    200
2018-05-01 16:32:45+00:00    200
2015-11-09 12:26:20+00:00    188
2018-05-01 16:33:35+00:00    164
2018-05-01 16:32:55+00:00    142
                            ... 
2021-11-17 12:37:13+00:00      1
2021-12-06 17:07:37+00:00      1
2021-11-16 20:37:06+00:00      1
2021-11-29 19:57:33+00:00      1
2020-11-13 03:42:20+00:00      1
Name: count, Length: 883366, dtype: int64

In [49]:
df.closeddate[df.closeddate == '9999'] = '9999-12-31'

df.closeddate.value_counts()

# df.closeddate.apply(pd.to_datetime) An error

closeddate
9999-12-31                   35107
2018-05-01 16:32:45+00:00      200
2018-05-01 16:32:02+00:00      200
2015-11-09 12:26:20+00:00      188
2018-05-01 16:33:35+00:00      164
                             ...  
2021-11-17 12:37:13+00:00        1
2021-12-06 17:07:37+00:00        1
2021-11-16 20:37:06+00:00        1
2021-11-29 19:57:33+00:00        1
2020-11-13 03:42:20+00:00        1
Name: count, Length: 883367, dtype: int64

found a solution here [here](https://www.statology.org/pandas-out-of-bounds-nanosecond-timestamp/)

In [53]:
df.closeddate[df.closeddate == '9999-12-31'] = pd.Timestamp.max

df.closeddate.value_counts()

closeddate
2262-04-11 23:47:16.854775807    35107
2018-05-01 16:32:45+00:00          200
2018-05-01 16:32:02+00:00          200
2015-11-09 12:26:20+00:00          188
2018-05-01 16:33:35+00:00          164
                                 ...  
2021-11-17 12:37:13+00:00            1
2021-12-06 17:07:37+00:00            1
2021-11-16 20:37:06+00:00            1
2021-11-29 19:57:33+00:00            1
2020-11-13 03:42:20+00:00            1
Name: count, Length: 883367, dtype: int64

In [58]:
# dropping all other nulls in the ds
print(f'Dropping {df.shape[0] -  df.dropna().shape[0]} rows when dropping the nulls')

df.dropna(inplace=True)

df.shape

Dropping 24008 rows when dropping the nulls


### Investigating nulls

In [ ]:
# closseddate invesigation
close_date_null = df[df['closeddate'].isna() == True] # getting the df

# looking at the statuses, if the status is cklosed and there is no closeddate then there is an issue
close_date_null['status'].value_counts() # all of the nulls in this section are open which is to be expected



status
Open    35107
Name: count, dtype: int64

In [ ]:
# checking the township null segments
summarize(df[df['township'].isna()])

## Dropping uneccessary columns and adding the time delta

In [127]:
df = get_mac_data()

# change columsn names to lower, remove the __c and that's it
col_rename = {}

for col in df.columns:
    col_rename[col] = col.lower().replace('__c', '').replace('date', '')

col_rename['LASTMODIFIEDDATE'] = 'last_modified'

df.rename(col_rename, axis=1, inplace = True)

{'OBJECTID': 'objectid', 'CASENUMBER': 'casenumber', 'SOURCE_ID__C': 'source_id', 'KEYWORD__C': 'keyword', 'SUBCATEGORY__C': 'subcategory', 'INCIDENT_ADDRESS__C': 'incident_address', 'TOWNSHIP__C': 'township', 'CITY__C': 'city', 'ZIP__C': 'zip', 'COUNCIL_DISTRICT__C': 'council_district', 'CREATEDDATE': 'created', 'LASTMODIFIEDDATE': 'last_modified', 'CLOSEDDATE': 'closed', 'STATUS': 'status', 'ORIGIN': 'origin'}


In [116]:
# solution to out of bound and tz naive/tz aware discrepancy
pd.Timestamp.max.floor('30D').tz_localize('US/Eastern')

Timestamp('2262-03-31 00:00:00-0500', tz='US/Eastern')

In [119]:
# df[['createddate','lastmodifieddate', 'closeddate']] = df[['createddate','lastmodifieddate', 'closeddate']].apply(pd.to_datetime)

df['close_time'] = df['closeddate'] - df['createddate']

df.head()

,source_id,keyword,subcategory,incident_address,township,city,zip,council_district,createddate,lastmodifieddate,closeddate,status,origin,close_time
0,INV22-01609,Illegal Dumping and Junk/Trash,Trash Accumulation or Dumped Materials,1142 N GOODLET AVE,WAYNE,INDIANAPOLIS,46222,11.0,2022-01-31 15:25:28+00:00,2022-02-01 13:59:02+00:00,2022-02-01 13:59:02+00:00,Closed,RequestIndy Mobile,0 days 22:33:34
1,A22-375189,Animal,Abuse,3501 RALSTON AVE,CENTER,INDIANAPOLIS,46218,9.0,2022-01-28 18:35:54+00:00,2022-01-29 03:43:27+00:00,2022-01-29 03:43:20+00:00,Closed,Phone,0 days 09:07:26
2,108343054,Trash,96 Gal Cart - Stolen,1102 N PARKER AVE,CENTER,INDIANAPOLIS,46201,17.0,2022-01-28 21:12:49+00:00,2022-03-21 14:07:36+00:00,2022-03-21 14:07:36+00:00,Closed,Phone,51 days 16:54:47
3,Email Sent,Abandoned Vehicle,Street/Alley (ABV),4631 ROSSLYN AVE,WASHINGTON,INDIANAPOLIS,46205,9.0,2022-01-30 21:23:42+00:00,2022-02-03 21:29:59+00:00,2022-02-03 21:29:59+00:00,Closed,RequestIndy Online,4 days 00:06:17
4,108343404,Trash,96 Gal Trash (Missed),4360 SAWYER AVE,LAWRENCE,INDIANAPOLIS,46226,13.0,2022-01-31 15:44:58+00:00,2022-01-31 16:37:08+00:00,2022-01-31 16:37:08+00:00,Closed,RequestIndy Online,0 days 00:52:10


In [ ]:
summarize(df)

In [24]:
df['createddate'].

dtype('O')

## Geocoding the mac data using geopandas
> **Resources:**
> * [Guidance used](https://autogis-site.readthedocs.io/en/2019/notebooks/L3/geocoding_in_geopandas.html)
> * [Geopy Documentation](https://geopy.readthedocs.io/en/stable/#)


In [4]:
# getting the clean mac data
df = clean_mac_data()

Dropping 24008 rows


In [ ]:
# gtting only the addresses
print(df.columns)

# getting geocode format addresses
df['full_address'] = df['incident_address'] + ' ' +  df['city'] + ' ' + df['state'] + ' ' + df['zip'].astype(str)

df.head(1)

In [5]:
# getting geocode tool
from geopandas.tools import geocode

In [5]:
# getting a small batch to try it out first
geo = geocode(df.iloc[1]['full_address'], provider='Nominatim', user_agent='indy_action_time', timeout=10)
geo
# success, but now we need to set it up for all of them

,geometry,address
0,POINT (-86.13089 39.81994),"3501, Ralston Avenue, Indianapolis, Marion Cou..."


### Rate Limiter using geopy

In [6]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from shapely.geometry import Point

In [7]:
# creating a smaller df to see if it works well
temp = pd.DataFrame(df.iloc[:5].loc[:,'full_address'])

In [8]:
# initiate geocoder 
geolocator = Nominatim(user_agent='indy_action_time')

# create a geopy rate limite:
geocode_with_delay = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# apply the geocoder with delay using the rate limiter
temp['temp'] = temp['full_address'].apply(geocode_with_delay)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('1142 N GOODLET AVE INDIANAPOLIS IN 46222',), **{}).
Traceback (most recent call last):
  File "/Users/aaron/codeup-data-science/env/lib/python3.9/site-packages/geopy/geocoders/base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "/Users/aaron/codeup-data-science/env/lib/python3.9/site-packages/geopy/adapters.py", line 447, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "/Users/aaron/codeup-data-science/env/lib/python3.9/site-packages/geopy/adapters.py", line 475, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 502

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/aaron/codeup-data-science/env/lib/python3.9/site-packages/geopy/extra/rate_limiter.py", line 136, in _retries_gen
    yield i  # Run t

In [9]:
temp

,full_address,temp
0,1142 N GOODLET AVE INDIANAPOLIS IN 46222,None
1,3501 RALSTON AVE INDIANAPOLIS IN 46218,None
2,1102 N PARKER AVE INDIANAPOLIS IN 46201,None
3,4631 ROSSLYN AVE INDIANAPOLIS IN 46205,None
4,4360 SAWYER AVE INDIANAPOLIS IN 46226,None


## Additions to the function based on explore 
* Changing the close time for open requests to todays date plus 1 day
    * Removed the closed dates bc the data is not update beginning in august
* A kwarg for the status will be added in aswell - This is for multivariate analysis
    * For this kwarg, if the staus section is dropped then the open cases will be dropped too
* This will be the dropping of certain columns such as incident_address
* Investigate the outliers from the response times and handle them approproately
    * Removed outliers
* Discretizing the response time will also be added
    * pdcut pandas generated bins and labeled

#### Changing the close time for open requests to todays date plus 1 day

In [8]:
df = get_mac_data()

# change columsn names to lower, remove the __c and that's it
col_rename = {}

for col in df.columns:
    col_rename[col] = col.lower().replace('__c', '').replace('date', '')

col_rename['LASTMODIFIEDDATE'] = 'last_modified'

df.rename(col_rename, axis=1, inplace = True)

# changing multiple columns to date time dtype
df[['created','last_modified', 'closed']] = df[['created','last_modified', 'closed']].apply(pd.to_datetime)

# filling in the closeddate nulls with the date the furthest out
df.closed.fillna(pd.to_datetime('today').normalize().tz_localize('US/Eastern'), inplace=True)

df.closed.hist()

#### Drop inicident address and change councli district to an object

In [14]:
# adding drop
df.columns

# dropping the unneeded columns
df.drop(['objectid', 'casenumber', 'source_id', 'incident_address'], axis=1, inplace=True)

df.head(1)

,keyword,subcategory,township,city,zip,council_district,created,last_modified,closed,status,origin
0,Illegal Dumping and Junk/Trash,Trash Accumulation or Dumped Materials,WAYNE,INDIANAPOLIS,46222,11.0,2022-01-31 15:25:28+00:00,2022-02-01 13:59:02+00:00,2022-02-01 13:59:02+00:00,Closed,RequestIndy Mobile


In [19]:
# changing council district dtype

# drop nulls
df = df.dropna()

# changing the type to string object
df['council_district'] = df.council_district.astype(int).astype(str)

#### KWARG: status
* becasue the most recent update for the data is from August - contacted the MAC service desk and sent an email to Information service Agency on Tuesday Aug 15 2023 for a year old issue

In [30]:
status = 'drop'

if status == 'drop':

    df = df[df.status == 'Closed']

    df.drop('status', axis=1, inplace=True)
    
elif status == 'keep':

    pass
    

#### Response time outliers and discretization

In [51]:
df.response_time.describe()

count                         882374
mean      47 days 01:55:58.844139786
std      208 days 14:47:12.869852328
min                  0 days 00:00:00
25%                  0 days 23:09:00
50%           3 days 12:47:42.500000
75%          10 days 07:20:16.750000
max               2563 days 13:01:40
Name: response_time, dtype: object

In [57]:
df[df.response_time > df.response_time.quantile(.75)].response_time.describe()

count                         220594
mean     180 days 10:24:01.978127240
std      387 days 18:08:09.797427092
min                 10 days 07:20:28
25%          18 days 16:44:49.500000
50%                 37 days 21:25:45
75%         114 days 01:09:54.500000
max               2563 days 13:01:40
Name: response_time, dtype: object

In [98]:
# (df.response_time / pd.Timedelta(weeks=1))

In [96]:
# df['created'], df['closed']

(0        2022-01-31 15:25:28+00:00
 1        2022-01-28 18:35:54+00:00
 3        2022-01-30 21:23:42+00:00
 4        2022-01-31 15:44:58+00:00
 5        2022-01-30 22:37:17+00:00
                     ...           
 940633   2020-10-28 13:41:38+00:00
 940634   2022-03-02 18:04:11+00:00
 940635   2021-06-23 15:58:24+00:00
 940636   2020-10-29 10:10:10+00:00
 940637   2020-10-29 13:17:42+00:00
 Name: created, Length: 772962, dtype: datetime64[ns, UTC],
 0        2022-02-01 13:59:02+00:00
 1        2022-01-29 03:43:20+00:00
 3        2022-02-03 21:29:59+00:00
 4        2022-01-31 16:37:08+00:00
 5        2022-02-03 22:44:59+00:00
                     ...           
 940633   2020-10-28 17:43:16+00:00
 940634   2022-03-09 19:08:30+00:00
 940635   2021-06-24 18:44:11+00:00
 940636   2020-10-29 18:12:11+00:00
 940637   2020-11-13 03:42:20+00:00
 Name: closed, Length: 772962, dtype: datetime64[ns, UTC])

In [97]:
# pd.Interval(df['created'],
# df['closed'], closed='left')

In [67]:
df = remove_outliers(df, k=3)

In [100]:
df['response_rating'] = pd.cut(df['response_time'], 5, labels=['excellent', 'great', 'good', 'fair','poor'])

df.head(1)

,keyword,subcategory,township,city,zip,council_district,created,last_modified,closed,origin,response_time,response_rating
0,Illegal Dumping and Junk/Trash,Trash Accumulation or Dumped Materials,WAYNE,INDIANAPOLIS,46222,11,2022-01-31 15:25:28+00:00,2022-02-01 13:59:02+00:00,2022-02-01 13:59:02+00:00,RequestIndy Mobile,0 days 22:33:34,excellent


In [112]:
pd.cut(df['response_time'], 5).value_counts()

response_time
(-1 days +23:04:48.418000, 7 days 15:58:36.400000]    625619
(7 days 15:58:36.400000, 15 days 07:57:12.800000]      76319
(15 days 07:57:12.800000, 22 days 23:55:49.200000]     33316
(22 days 23:55:49.200000, 30 days 15:54:25.600000]     20377
(30 days 15:54:25.600000, 38 days 07:53:02]            17331
Name: count, dtype: int64

In [111]:
df.response_rating.describe()

count        772962
unique            5
top       excellent
freq         625619
Name: response_rating, dtype: object

In [116]:
df.drop('response_time', axis=1)

,keyword,subcategory,township,city,zip,council_district,created,last_modified,closed,origin,response_rating
0,Illegal Dumping and Junk/Trash,Trash Accumulation or Dumped Materials,WAYNE,INDIANAPOLIS,46222,11,2022-01-31 15:25:28+00:00,2022-02-01 13:59:02+00:00,2022-02-01 13:59:02+00:00,RequestIndy Mobile,excellent
1,Animal,Abuse,CENTER,INDIANAPOLIS,46218,9,2022-01-28 18:35:54+00:00,2022-01-29 03:43:27+00:00,2022-01-29 03:43:20+00:00,Phone,excellent
3,Abandoned Vehicle,Street/Alley (ABV),WASHINGTON,INDIANAPOLIS,46205,9,2022-01-30 21:23:42+00:00,2022-02-03 21:29:59+00:00,2022-02-03 21:29:59+00:00,RequestIndy Online,excellent
4,Trash,96 Gal Trash (Missed),LAWRENCE,INDIANAPOLIS,46226,13,2022-01-31 15:44:58+00:00,2022-01-31 16:37:08+00:00,2022-01-31 16:37:08+00:00,RequestIndy Online,excellent
5,Abandoned Vehicle,Street/Alley (ABV),CENTER,INDIANAPOLIS,46202,11,2022-01-30 22:37:17+00:00,2022-02-03 22:44:59+00:00,2022-02-03 22:44:59+00:00,RequestIndy Online,excellent
...,...,...,...,...,...,...,...,...,...,...,...
940633,Animal,Noise (Animal),CENTER,INDIANAPOLIS,46201,12,2020-10-28 13:41:38+00:00,2020-10-28 17:43:21+00:00,2020-10-28 17:43:16+00:00,Phone,excellent
940634,Chuckhole,Street (Chuckhole),DECATUR,INDIANAPOLIS,46221,20,2022-03-02 18:04:11+00:00,2022-03-09 19:08:30+00:00,2022-03-09 19:08:30+00:00,Phone,excellent
940635,Weeds,Private Property (Complaint),CENTER,INDIANAPOLIS,46201,12,2021-06-23 15:58:24+00:00,2021-06-24 18:44:11+00:00,2021-06-24 18:44:11+00:00,Phone,excellent
940636,Debris/Litter,Debris in Alley/Street,CENTER,INDIANAPOLIS,46208,9,2020-10-29 10:10:10+00:00,2020-12-03 20:47:07+00:00,2020-10-29 18:12:11+00:00,RequestIndy Mobile,excellent


0         (-1 days +23:04:48.418000, 7 days 15:58:36.400...
1         (-1 days +23:04:48.418000, 7 days 15:58:36.400...
3         (-1 days +23:04:48.418000, 7 days 15:58:36.400...
4         (-1 days +23:04:48.418000, 7 days 15:58:36.400...
5         (-1 days +23:04:48.418000, 7 days 15:58:36.400...
                                ...                        
940633    (-1 days +23:04:48.418000, 7 days 15:58:36.400...
940634    (-1 days +23:04:48.418000, 7 days 15:58:36.400...
940635    (-1 days +23:04:48.418000, 7 days 15:58:36.400...
940636    (-1 days +23:04:48.418000, 7 days 15:58:36.400...
940637    (7 days 15:58:36.400000, 15 days 07:57:12.800000]
Length: 772962, dtype: category
Categories (5, interval[timedelta64[ns], right]): [(-1 days +23:04:48.418000, 7 days 15:58:36.400... < (7 days 15:58:36.400000, 15 days 07:57:12.800000] < (15 days 07:57:12.800000, 22 days 23:55:49.200... < (22 days 23:55:49.200000, 30 days 15:54:25.600... < (30 days 15:54:25.600000, 38 days 07:53:02]]

##### Note:
* I think the response time ratings may need to be calculated based on the department, but this is something for later - put in the trello idra bank

Note on reponse_levels:
* (-1 days +23:04:48.418000, 7 days 15:58:36.400000]    625619
* (7 days 15:58:36.400000, 15 days 07:57:12.800000]      76319
* (15 days 07:57:12.800000, 22 days 23:55:49.200000]     33316
* (2 days 23:55:49.200000, 30 days 15:54:25.600000]     20377
* (0 days 15:54:25.600000, 38 days 07:53:02]            17331
* Name: count, dtype: int64

# Creating the function

In [60]:
def remove_outliers(df, k=1.5):
    '''
    Actions: removies outliers using the IQR with a default k of 1.5
    '''
    # initialize dictionary
    col_qs = {}
    
    # assign column names to variable
    df_cols = df.columns
    
    # creates a list of column names
    df_cols = df_cols.to_list()
    
    # remove cat cols
    cat_cols = cat_cols = [col for col in df.columns if df[col].dtype == 'O']

    for col in cat_cols:
        df_cols.remove(col)

    # for each column
    for col in df_cols:
        
        # create qualtiles and put them in a dict
        col_qs[col] = q1, q3 = df[col].quantile([0.25, 0.75])

    # for each col
    for col in df_cols:    
        
        # calculate the iqr
        iqr = col_qs[col][0.75] - col_qs[col][0.25]
        
        # calculate the lower fence
        lower_fence = col_qs[col][0.25] - (iqr*k)
        
        # calculates the upper fence
        upper_fence = col_qs[col][0.75] + (iqr*k)
        
        # remove outliers from df for each column
        df = df[(df[col] > lower_fence) & (df[col] < upper_fence)]
        
    # exit df and return new df
    return df

In [127]:
def get_clean_mac(status_focus = 'closed'):
    '''
    This function acquires mac data, renames columns, changes dtypes, fills in null values for the closed date, drops all other null values,
    drops objectid, caseneumber, and source_id, and creates a new columns
    KWARG: status_focus: 'closed' drops all open status cases and the status column
    Modules:
        from acquire import get_mac_data
        import pandas as pd
    '''
    # get data
    df = get_mac_data()

    # change columsn names to lower, remove the __c and that's it
    col_rename = {}

    for col in df.columns:
       
        col_rename[col] = col.lower().replace('__c', '').replace('date', '')

    col_rename['LASTMODIFIEDDATE'] = 'last_modified'

    df.rename(col_rename, axis=1, inplace = True)

    # changing multiple columns to date time dtype
    df[['created','last_modified', 'closed']] = df[['created','last_modified', 'closed']].apply(pd.to_datetime)

    # dropping null values
    # print(f'Dropping {df.shape[0] - df.dropna().shape[0]} rows')
    df.dropna(inplace=True)

    # KWARG if we only focus on closed status
    if status_focus == 'closed':

        # keep only closed status cases
        df = df[df.status == 'Closed']

        # and drop the status col bc no variance
        df.drop('status', axis=1, inplace=True)    
        
    # KWARG if we want open - keep it the same
    elif status_focus == 'open': 
    
        pass

    # dropping the unneeded columns
    df.drop(['objectid', 'casenumber', 'source_id', 'incident_address'. 'last_modified'], axis=1, inplace=True)

    # changing dtype to string object
    df['council_district'] = df.council_district.astype(int).astype(str)
    df['zip'] = df.zip.astype(str)

    # adding the time from creation to closing
    df['response_time'] = df['closed'] - df['created']
    
    # removing outliers
    df = remove_outliers(df, k=3)
    
    # created the reponse rating
    df['response_rating'] = pd.cut(df['response_time'], 5, labels=['excellent', 'great', 'good', 'fair','poor'])

    # dropping time related columns
    df.drop(['response_time', 'created', 'closed'], axis=1, inplace=True)
    
    return df

In [128]:
df = get_clean_mac()

df.shape

(772962, 11)

## Removed from function

In [ ]:
# adding the state
# df['state'] = 'IN'

# getting geocode format addresses
# df['full_address'] = df['incident_address'] + ' ' +  df['city'] + ' ' + df['state'] + ' ' + df['zip'].astype(str)

# filling in the closeddate nulls with the date the furthest out
# df.closed.fillna(pd.to_datetime('today').normalize().tz_localize('US/Eastern'), inplace=True)

#### Notes:
* STATUS: is it useful to use the open status information? I don't think so

#### Action:
* Removing all open status claims
    * The open status claims all have no closeddate which means we cannot use this data to train our model on. 
    * This data can be useful for the identifyinf which type of service requests are not handled for extedned periods of time, but that should be addressed in the the data already available unless there are some requests that have never closed even when introduced previosuly. That seems like a special case experience whoc deserves its own investigation